In [29]:
import pandas as pd
from sqlalchemy import create_engine
from geopy.geocoders import Nominatim
import time
from pprint import pprint
import geopy.distance
from random import random
from time import sleep
from random import randint
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

In [14]:
disaster_data = "static/data/DisasterDeclarationsSummaries.csv"
disaster_df = pd.read_csv(disaster_data)
disaster_df.drop(['Unnamed: 25', 'Unnamed: 26','Unnamed: 27'], axis=1, inplace=True)
disaster_df.head()

,femaDeclarationString,disasterNumber,state,state.1,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,...,disasterCloseoutDate,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,hash,lastRefresh,id,countyState
0,DR-4607-MI,4607,MI,Michigan,DR,2021-07-15T10:41:00.000Z,2021,Severe Storms,"SEVERE STORMS, FLOODING, AND TORNADOES",1,...,NaN,26,163,99163,Wayne County,21051,6862d74686de9eaea31be8c0d33a0051867f674c,2021-07-17T02:42:14.846Z,60f243865d037ccb7d217dbd,"Wayne County, Michigan"
1,DR-4607-MI,4607,MI,Michigan,DR,2021-07-15T10:41:00.000Z,2021,Severe Storms,"SEVERE STORMS, FLOODING, AND TORNADOES",1,...,NaN,26,161,99161,Washtenaw County,21051,c9f45069cae6640e009712d996cd8481e7b081f0,2021-07-17T02:42:14.846Z,60f243865d037ccb7d217dba,"Washtenaw County, Michigan"
2,FM-5398-WA,5398,WA,Washington,FM,2021-07-14T18:36:00.000Z,2021,Fire,RED APPLE FIRE,0,...,NaN,53,0,0,Statewide,21054,0a1f400f649de3ab95d21912ded0f302c7473449,2021-07-17T02:42:15.243Z,60f243875d037ccb7d2181eb,Washington
3,FM-5397-WA,5397,WA,Washington,FM,2021-07-13T21:27:00.000Z,2021,Fire,CHUWEAH CREEK FIRE,0,...,NaN,53,0,14240,Colville Indian Reservation,21052,5f811f571c49e7b0ac2763d00fa633afa360b4e7,2021-07-17T02:42:15.242Z,60f243875d037ccb7d2181e7,"Colville Indian Reservation, Washington"
4,FM-5396-OR,5396,OR,Oregon,FM,2021-07-10T20:22:00.000Z,2021,Fire,BOOTLEG FIRE,0,...,NaN,41,35,99035,Klamath County,21053,d5f2cd7a57557a2f7e245570c783bc293a14b530,2021-07-17T02:42:15.241Z,60f243875d037ccb7d2181e5,"Klamath County, Oregon"


In [32]:
disaster_summary_df = disaster_df.loc[disaster_df["fyDeclared"]>2010]
disaster_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22145 entries, 0 to 22144
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   femaDeclarationString     22145 non-null  object
 1   disasterNumber            22145 non-null  int64 
 2   state                     22145 non-null  object
 3   state.1                   22145 non-null  object
 4   declarationType           22145 non-null  object
 5   declarationDate           22145 non-null  object
 6   fyDeclared                22145 non-null  int64 
 7   incidentType              22145 non-null  object
 8   declarationTitle          22145 non-null  object
 9   ihProgramDeclared         22145 non-null  int64 
 10  iaProgramDeclared         22145 non-null  int64 
 11  paProgramDeclared         22145 non-null  int64 
 12  hmProgramDeclared         22145 non-null  int64 
 13  incidentBeginDate         22145 non-null  object
 14  incidentEndDate       

In [33]:
app = Nominatim(user_agent="tutorial")

In [34]:
def get_lat(countyState):
    try:
        lat = app.geocode(countyState,timeout=10000).raw["lat"]
        lon = app.geocode(countyState,timeout=10000).raw["lon"]
        #print(f'{countyState} lat = {lat}, lon = {lon}')
        return lat,lon
    except GeocoderTimedOut:
        logging.info('TIMED OUT: GeocoderTimedOut: Retrying...')
        sleep(randint(1*100,sleep_sec*100)/100)
        lat = app.geocode(countyState,timeout=10000).raw["lat"]
        lon = app.geocode(countyState,timeout=10000).raw["lon"]
        return lat,lon
    except (TypeError, AttributeError):
        lat = "unknown"
        lon = "unknown"
        #print(f'{countyState} not found')
        return lat, lon

In [19]:
#get_lat("Clay County, Indiana")

In [35]:
disaster_summary_df["lat"],disaster_summary_df["lon"]  = zip(*disaster_summary_df["countyState"].apply(get_lat))

<ipython-input-35-4eeb50dc43c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disaster_summary_df["lat"],disaster_summary_df["lon"]  = zip(*disaster_summary_df["countyState"].apply(get_lat))


In [41]:
disaster_summary_df = disaster_summary_df[disaster_summary_df["lat"] != "unknown"]

In [40]:
disaster_summary_df.to_csv("static/data/disasterData.csv")